# Lion Cheetah Classification task

Baed on the following Kaggle data set: https://www.kaggle.com/datasets/mikoajfish99/lions-or-cheetahs-image-classification

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow
import cv2
import os

In [6]:
# In this case, we have two data sets separting cheetah and lions images

cheetah_folder = "./images/Cheetahs"
lion_folder = "./images/Lions"

def load_images_from_folder(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        if os.path.isfile(img_path):
            image = cv2.imread(img_path)
            if image is not None:
                images.append(image)
                labels.append(folder.split('/')[-1])  # Assuming folder name is the label
    return images, labels

cheetah_images, cheetah_labels = load_images_from_folder(cheetah_folder)
lion_images, lion_labels = load_images_from_folder(lion_folder)

In [12]:
cheetah_images

[array([[[ 15,  76,  72],
         [ 38,  99,  95],
         [ 37, 103,  98],
         ...,
         [  0,  64,  45],
         [  0,  60,  49],
         [  0,  59,  49]],
 
        [[ 40, 111, 101],
         [ 56, 123, 114],
         [ 52, 115, 105],
         ...,
         [  4,  65,  45],
         [  1,  60,  46],
         [  2,  58,  47]],
 
        [[ 15,  73,  62],
         [ 11,  69,  58],
         [  6,  67,  53],
         ...,
         [  4,  63,  43],
         [  0,  57,  42],
         [  2,  56,  43]],
 
        ...,
 
        [[ 94, 148, 159],
         [ 78, 131, 144],
         [ 70, 122, 135],
         ...,
         [ 75, 125, 137],
         [ 87, 136, 146],
         [ 70, 117, 125]],
 
        [[ 94, 144, 162],
         [ 82, 130, 148],
         [ 72, 119, 140],
         ...,
         [ 86, 132, 150],
         [ 91, 138, 152],
         [ 80, 125, 139]],
 
        [[ 92, 142, 162],
         [ 84, 131, 153],
         [ 75, 119, 142],
         ...,
         [ 85, 131, 149],
  